In [45]:
from pyspark.sql.functions import *

StatementMeta(sparkpool, 13, 34, Finished, Available, Finished)

In [46]:
from notebookutils import mssparkutils

StatementMeta(sparkpool, 13, 35, Finished, Available, Finished)

In [47]:
dateFormat = "%Y-%m-%d"

import datetime

ts = spark.sql("select current_timestamp() as ctime").collect()[0]["ctime"]

todaydate = ts.strftime(dateFormat)

StatementMeta(sparkpool, 13, 36, Finished, Available, Finished)

In [48]:
sourceaccountname = "storageottonprem011"
sourcecontainer = "raw"
sourcelinkedservice = "ls_adls"
sourcefilelocation = "ingest/"+todaydate
apifilelocation = todaydate+".csv"

StatementMeta(sparkpool, 13, 37, Finished, Available, Finished)

In [49]:

source_full_storage_account_name = "storageottonprem011.dfs.core.windows.net"
spark.conf.set("spark.storage.synapse.linkedServiceName", f"{sourcelinkedservice}")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{source_full_storage_account_name}", "com.microsoft.azure.synapse.tokenlibrary.LinkedServiceBasedTokenProvider")


StatementMeta(sparkpool, 13, 38, Finished, Available, Finished)

In [50]:
file = f"abfss://{sourcecontainer}@{sourceaccountname}.dfs.core.windows.net/{sourcefilelocation}"
apifile = f"abfss://{sourcecontainer}@{sourceaccountname}.dfs.core.windows.net/api/{apifilelocation}"
print(file)
print(apifile)

StatementMeta(sparkpool, 13, 39, Finished, Available, Finished)

abfss://raw@storageottonprem011.dfs.core.windows.net/ingest/2024-12-01
abfss://raw@storageottonprem011.dfs.core.windows.net/api/2024-12-01.csv


In [52]:
try:
    source_df = spark.read.format('csv').option('header','true').option('inferschema','true').load(file)
except Exception as e:
    print(f"File does not exist : {e}")



StatementMeta(sparkpool, 13, 41, Finished, Available, Finished)

In [53]:
try:

    nodeups_df = source_df

    nodeups_df = nodeups_df.drop_duplicates()

    nodeups_df = nodeups_df.na.drop(how='all')

    df_nonulls = nodeups_df.na.fill(value='unknown')

    df_nonulls = df_nonulls.withColumn("IMDB category",\
        when(col("IMDB Score").between(1,2.9),"Very Low")\
        .when(col("IMDB Score").between(3,4.9),"Low")\
        .when(col("IMDB Score").between(5,7.9),"Medium")\
        .otherwise("High")\
        )

    df = df_nonulls.withColumn("Runtime Hours"\
        , round(df_nonulls.Runtime/60,2)
        )

    df = df.drop("Runtime")

    df = df.withColumn('ReleaseDate',to_date('ReleaseDate','dd-MM-yyyy'))
    df = df.withColumn('AddedDate',to_date('AddedDate','dd-MM-yyyy'))
    df = df.withColumnRenamed("IMDB Score", "IMDB_Score")
    df = df.withColumnRenamed("IMDB category", "IMDB_category")
    df = df.withColumnRenamed("Runtime Hours", "Runtime_hours")
except Exception as e:
    print("source_df does not exist")

StatementMeta(sparkpool, 13, 42, Finished, Available, Finished)

In [54]:
display(df)

StatementMeta(sparkpool, 13, 43, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, edf1e8c1-9d6c-4558-bdff-a78494c43ede)

In [55]:
destaccountname = "storageottonprem011"
destcontainer = "refined"
destfolder = "data/"

StatementMeta(sparkpool, 13, 44, Finished, Available, Finished)

In [56]:
#writing data to adls
try:
    destpath = f"abfss://{destcontainer}@{destaccountname}.dfs.core.windows.net/data"

    df.write\
    .mode("append")\
    .format("parquet")\
    .option("header", "true")\
    .option("Inferschema", "true")\
    .save(destpath)
except Exception as e:
    print("df does not exist")

StatementMeta(sparkpool, 13, 45, Finished, Available, Finished)